# Import thư viện

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import traceback

In [12]:
# Mở trình duyệt Google Chrome ở trang thái Tab ẩn danh
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument("--start-maximized")
s = Service('C:\chromedriver')
driver = webdriver.Chrome(service=s, options=options)
driver.implicitly_wait(0)

In [13]:
# List tất cả url movie
list_all_url_movie = pd.read_csv('E:/Crawl_Web/Rotten Tomatoes/Crawl URL/URL_movie.csv').squeeze('columns').tolist()

In [14]:
df_movie = []

for url_movie in list_all_url_movie:
    try:
        # Mở url phim
        driver.get(url_movie)
        
        # Đợi cho đến khi url load xong
        element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'main[id="main_container"]')))
        html_of_interest = driver.execute_script('return arguments[0].innerHTML',element)
        soup = BeautifulSoup(html_of_interest, 'lxml')

        ## Tạo Dict chứa thông tin về phim
        dict_info_movie = {}

        # Tiêu đề phim
        dict_info_movie['Title'] = soup.select('h1[class="scoreboard__title"]')[0].text

        # Điểm đánh giá của nhà phê bình
        tomatometer = soup.select('score-board')[0].attrs['tomatometerscore']
        # if tomatometer != '':
        dict_info_movie['Tomatometer score'] = tomatometer

        # Điểm đánh giá của khán giả
        dict_info_movie['Audience score'] = soup.select('score-board')[0].attrs['audiencescore']

        # Số lượt đánh giá của nhà phê bình
        dict_info_movie['Tomatometer count'] = soup.select('a[slot="critics-count"]')[0].text.strip()

        # Số lượt đánh giá của khán giả
        dict_info_movie['Audience count'] = soup.select('a[slot="audience-count"]')[0].text.strip()

        # Trạng thái đánh giá phim của nhà phê bình
        dict_info_movie['Tomatometer state'] = soup.select('score-board')[0].attrs['tomatometerstate']

        # Trạng thái đánh giá phim của khán giả
        dict_info_movie['Audience state'] = soup.select('score-board')[0].attrs['audiencestate']

        # Thông tin phim
        list_name_info = soup.select('b[class="info-item-label"]')
        list_info = soup.select('span[data-qa="movie-info-item-value"]')

        for num_info in range(len(list_name_info)):
            name_info = list_name_info[num_info].text.replace('  ', '').replace('\n', '').replace(':', '')
            info = list_info[num_info].text.replace('  ', '').replace('\n', '').replace(':', '')
            dict_info_movie[name_info] = info

        # Tóm tắt nội dụng phim
        dict_info_movie['Synopsis'] = soup.select('p[data-qa="movie-info-synopsis"]')[0].text.strip()

        # Diễn viên
        list_raw_cast = soup.select('a[data-qa="cast-crew-item-link"] > p')
        list_cast = []

        for c in range(len(list_raw_cast)):
            list_cast.append(list_raw_cast[c].text)

        dict_info_movie['Cast'] = ', '.join(list_cast)

        df_movie.append(dict_info_movie)

        # Reviews của nhà phê bình
        driver.get(url_movie + '/reviews')

        element_critic_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="layout reviews-page-container"]')))
        html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_critic_review)
        soup_critic_review = BeautifulSoup(html_of_interest, 'lxml')

        # Check có review của nhà phê bình không ?
        check_critic_review = soup_critic_review.select('div[class="review_table"]')[0].text.strip()

        if check_critic_review != '':
            list_critic_review = []
            list_top_critic_review = []
            
            # Số lượng review trong 1 trang
            num_rv = len(soup_critic_review.select('div[class="review-row"]'))

            for num in range(num_rv):
                rv = soup_critic_review.select('div[class="review-row"]')[num].find_all('p', {'class': 'review-text'})[0].text
                list_critic_review.append(rv)

                # Check top critic reviews
                check_top_critic = soup_critic_review.select('div[class="review-row"]')[num].find_all('rt-icon-top-critic')

                if check_top_critic != []:
                    list_top_critic_review.append(rv)
            
            while True:
                sleep(1)
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, 'rt-button[class="js-prev-next-paging-next"]')
                    driver.execute_script("arguments[0].click();", next_button)
                    
                    element_critic_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="review_table"]')))
                    html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_critic_review)
                    soup_critic_review = BeautifulSoup(html_of_interest, 'lxml')

                    num_rv = len(soup_critic_review.select('div[class="review-row"]'))

                    for num in range(num_rv):
                        rv = soup_critic_review.select('div[class="review-row"]')[num].find_all('p', {'class': 'review-text'})[0].text.strip()
                        list_critic_review.append(rv)

                        check_top_critic = soup_critic_review.select('div[class="review-row"]')[num].find_all('rt-icon-top-critic')

                        if check_top_critic != []:
                            list_top_critic_review.append(rv)
                    
                except:
                    break
            dict_info_movie['Critic review'] = '<>'.join(list_critic_review)
            if list_top_critic_review != []:
                dict_info_movie['Top critic review'] = '<>'.join(list_top_critic_review)
            else:
                dict_info_movie['Top critic review'] = 'No top critic reviews'
        elif check_critic_review == '':
            dict_info_movie['Critic review'] = 'No critic reviews'
            dict_info_movie['Top critic review'] = 'No top critic reviews'

        # Reviews của khán giả
        driver.get(url_movie + '/reviews?type=user')

        element_audience_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-qa="reviews-container"]')))
        html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_audience_review)
        soup_audience_review = BeautifulSoup(html_of_interest, 'lxml')

        ## Check có review của khán giả không ?
        check_audience_review = soup_audience_review.select('div[id="movieUserReviewsContent"]')

        if len(check_audience_review) != 0:
            list_audience_review = []
            list_verified_audience_review = []
            
            # Số lượng review trong 1 trang
            num_audience_rv = len(soup_audience_review.select('div[class="audience-review-row"]'))
            
            for num in range(num_audience_rv):
                rv = soup_audience_review.select('div[class="audience-review-row"]')[num].find_all('p', {'data-qa': 'review-text'})[0].text
                list_audience_review.append(rv)

                # Check verified audience review
                check_verified_audience = soup_audience_review.select('div[class="audience-review-row"]')[num].find_all('span', {'class': 'audience-reviews__verified'})

                if check_verified_audience != []:
                    list_verified_audience_review.append(rv)
            
            while True:
                sleep(1)
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, 'rt-button[class="js-prev-next-paging-next"]')
                    driver.execute_script("arguments[0].click();", next_button)
                    
                    element_audience_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="movieUserReviewsContent"]')))
                    html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_audience_review)
                    soup_audience_review = BeautifulSoup(html_of_interest, 'lxml')

                    # Số lượng review trong 1 trang
                    num_audience_rv = len(soup_audience_review.select('div[class="audience-review-row"]'))
                    
                    for num in range(num_audience_rv):
                        rv = soup_audience_review.select('div[class="audience-review-row"]')[num].find_all('p', {'data-qa': 'review-text'})[0].text
                        list_audience_review.append(rv)

                        # Check verified audience review
                        check_verified_audience = soup_audience_review.select('div[class="audience-review-row"]')[num].find_all('span', {'class': 'audience-reviews__verified'})

                        if check_verified_audience != []:
                            list_verified_audience_review.append(rv)
                    
                except:
                    break
            dict_info_movie['Audience review'] = '<>'.join(list_audience_review)
            if list_verified_audience_review != []:
                dict_info_movie['Verified audience review'] = '<>'.join(list_verified_audience_review)
            else:
                dict_info_movie['Verified audience review'] = 'No verified audience reviews'

        else:
            dict_info_movie['Audience review'] = 'No audience reviews'
            dict_info_movie['Verified audience review'] = 'No verified audience reviews'

        df_movie.append(dict_info_movie)

    except Exception:
        traceback.print_exc()
        print(url_movie)
        continue
    
df_movie = pd.DataFrame(df_movie)

Traceback (most recent call last):
  File "C:\Users\Nguyen Quang Tu\AppData\Local\Temp\ipykernel_19676\3074276906.py", line 17, in <module>
    dict_info_movie['Title'] = soup.select('h1[class="scoreboard__title"]')[0].text
IndexError: list index out of range


https://www.rottentomatoes.com/m/1010695-invisible_man


Traceback (most recent call last):
  File "C:\Users\Nguyen Quang Tu\AppData\Local\Temp\ipykernel_19676\3074276906.py", line 17, in <module>
    dict_info_movie['Title'] = soup.select('h1[class="scoreboard__title"]')[0].text
IndexError: list index out of range


https://www.rottentomatoes.com/m/saturday_night_fever_40th_anniversary


Traceback (most recent call last):
  File "C:\Users\Nguyen Quang Tu\AppData\Local\Temp\ipykernel_19676\3074276906.py", line 17, in <module>
    dict_info_movie['Title'] = soup.select('h1[class="scoreboard__title"]')[0].text
IndexError: list index out of range


...

https://www.rottentomatoes.com/m/passion_and_paradise


Traceback (most recent call last):
  File "C:\Users\Nguyen Quang Tu\AppData\Local\Temp\ipykernel_19676\3074276906.py", line 17, in <module>
    dict_info_movie['Title'] = soup.select('h1[class="scoreboard__title"]')[0].text
IndexError: list index out of range


https://www.rottentomatoes.com/m/green_mountain_film_festival


Traceback (most recent call last):
  File "C:\Users\Nguyen Quang Tu\AppData\Local\Temp\ipykernel_19676\3074276906.py", line 17, in <module>
    dict_info_movie['Title'] = soup.select('h1[class="scoreboard__title"]')[0].text
IndexError: list index out of range


https://www.rottentomatoes.com/m/met_summer_encore_der_rosenkavalier


In [15]:
df_movie

,Title,Tomatometer score,Audience score,Tomatometer count,Audience count,Tomatometer state,Audience state,Genre,Original Language,Director,...,Synopsis,Cast,Release Date (Streaming),Box Office (Gross USA),Rating,Release Date (Theaters),Distributor,Sound Mix,Aspect Ratio,View the collection
0,Roaring 20's,89,,9 Reviews,0 Ratings,fresh,,Drama,French (France),Elisabeth Vogler,...,On one beautiful afternoon in Paris during the...,"Noémie Schmidt, Alice De Lencquesaing, Manuel ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Baahubali: The Beginning,90,86,20 Reviews,"1,000+ Ratings",fresh,upright,"Adventure, Action, Drama",Telugu,S.S. Rajamouli,...,"In the kingdom of Mahishmati, while pursuing h...","Prabhas, Rana Daggubati, Anushka Shetty, Taman...","Aug 18, 2016",$7.7M,NaN,NaN,NaN,NaN,NaN,NaN
2,One Floor Below,83,72,23 Reviews,Fewer than 50 Ratings,fresh,upright,Drama,Romanian,Radu Muntean,...,Patrascu finds himself at odds with two neighb...,"Teo Corban, Maria Popistasu, Iulian Postelnicu...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Indian Summer,58,65,19 Reviews,"5,000+ Ratings",rotten,upright,"Comedy, Drama",English,Mike Binder,...,"Unca Lou (Alan Arkin), the longtime director o...","Alan Arkin, Matt Craven, Diane Lane, Bill Paxt...","Sep 3, 2002",$14.7M,PG-13,"Apr 23, 1993 wide","Touchstone Pictures, Buena Vista Pictures, BV ...",Surround,NaN,NaN
4,A Lion Returns,100,,5 Reviews,0 Ratings,fresh,,Drama,English,Serhat Caradee,...,Jamal has just returned to Australia after spe...,"Serhat Caradee, Tyler De Nawi, Danny Elacci, B...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5136,Chromophobia,31,53,13 Reviews,500+ Ratings,rotten,spilled,Drama,English,Martha Fiennes,...,Encouraged by his editor to seek 'sexy stories...,"Ralph Fiennes, Penélope Cruz, Kristin Scott Th...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5137,American Gun,40,41,20 Reviews,500+ Ratings,rotten,spilled,Drama,English,Alan Jacobs,...,A distraught man (James Coburn) embarks on a c...,"James Coburn, Barbara Bain, Virginia Madsen, A...","Feb 17, 2004",NaN,R,NaN,NaN,"Dolby Stereo, Dolby Digital, Dolby A, Surround...",Flat (1.851),NaN
5138,Tchoupitoulas,83,68,18 Reviews,250+ Ratings,fresh,upright,Documentary,English,"Bill Ross IV, Turner Ross",...,Three brothers discover the late-night scene o...,"Bill Ross IV, Turner Ross, Bill Ross IV, Turne...","Oct 14, 2013",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5139,Bag It,100,81,5 Reviews,100+ Ratings,fresh,upright,"Documentary, Comedy, Drama",English,Suzan Beraza,...,"Jeb Berrier, a regular American man, makes a p...","Suzan Beraza, Michelle Curry Wright`, Michelle...","Nov 18, 2016",NaN,NaN,"Sep 1, 2011 original",APT Worldwide,NaN,NaN,NaN


In [16]:
df_movie.to_csv('120000_end_movie_info.csv', index=False)